In [ ]:
import polars as pl
import numpy as np
from pathlib import Path

# Configs
DATA_PATH = Path("./data")
OUTPUT_PATH = Path("./feature_repo/data")
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

In [3]:
print("🚀 Iniciando ETL de Features...")

q = pl.scan_parquet(DATA_PATH / "transactions_train.parquet")

🚀 Iniciando ETL de Features...


In [5]:
# 2. Engenharia de Features de USUÁRIO (User Features)
# Exemplo: Comportamento histórico do usuário
print("⚙️ Calculando User Features...")
user_features_query = (
    q.group_by("customer_id")
    .agg([
        pl.col("price").mean().alias("avg_spend"),
        pl.col("price").count().alias("purchase_count"),
        pl.col("price").sum().alias("total_spend"),
        pl.col("t_dat").max().alias("last_purchase_date")
    ])
    .with_columns(pl.col("last_purchase_date").alias("event_timestamp"))
)

print("⚙️ Executando agregação de Usuários (Streaming)...")
user_features = user_features_query.collect(streaming=True)

print(f"💾 Salvando {user_features.shape[0]} features de usuário...")
user_features.write_parquet(OUTPUT_PATH / "user_features.parquet")

⚙️ Calculando User Features...
⚙️ Executando agregação de Usuários (Streaming)...


/tmp/ipykernel_2055/688410323.py:16: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  user_features = user_features_query.collect(streaming=True)


💾 Salvando 1362281 features de usuário...


In [8]:
user_features.head()

customer_id,avg_spend,purchase_count,total_spend,last_purchase_date,event_timestamp
str,f32,u32,f32,datetime[ns],datetime[ns]
"""1ffac95eaa5b45593647ce7da6abd1…",0.032585,51,1.661847,2020-09-19 00:00:00,2020-09-19 00:00:00
"""2441f7cc909e104eb1f93956de0f9b…",0.031438,29,0.911712,2020-09-02 00:00:00,2020-09-02 00:00:00
"""25feddb8085cdcbcbe2bd42f159f95…",0.028787,35,1.007542,2020-07-15 00:00:00,2020-07-15 00:00:00
"""28498c10b69acf435d1b51c06aa7b8…",0.031827,139,4.423932,2020-09-05 00:00:00,2020-09-05 00:00:00
"""2994983e23861ac1d3196c52a3fc71…",0.023405,11,0.257458,2020-07-18 00:00:00,2020-07-18 00:00:00


In [9]:
del user_features

In [11]:
# 2. Engenharia de Features de USUÁRIO (User Features)
# Exemplo: Comportamento histórico do usuário
print("⚙️ Calculando Item Features...")
item_features_query = (
    q.group_by("article_id")
    .agg([
        pl.col("customer_id").count().alias("popularity_score"),
        pl.col("price").mean().alias("avg_price"),
        pl.col("t_dat").max().alias("last_sold_date")
    ])
    .with_columns(pl.col("last_sold_date").alias("event_timestamp"))
)

print("⚙️ Executando agregação de Usuários (Streaming)...")
item_features = item_features_query.collect(streaming=True)

print(f"💾 Salvando {item_features.shape[0]} features de usuário...")
item_features.write_parquet(OUTPUT_PATH / "item_features.parquet")
print("✅ ETL Finalizado com sucesso usando Polars!")

⚙️ Calculando Item Features...
⚙️ Executando agregação de Usuários (Streaming)...


/tmp/ipykernel_2055/2572177188.py:15: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  item_features = item_features_query.collect(streaming=True)


💾 Salvando 104547 features de usuário...
✅ ETL Finalizado com sucesso usando Polars!


In [12]:
item_features.head()

article_id,popularity_score,avg_price,last_sold_date,event_timestamp
str,u32,f32,datetime[ns],datetime[ns]
"""0721991003""",276,0.012193,2019-03-30 00:00:00,2019-03-30 00:00:00
"""0523490003""",3323,0.004197,2020-09-21 00:00:00,2020-09-21 00:00:00
"""0660712014""",1327,0.037222,2020-01-02 00:00:00,2020-01-02 00:00:00
"""0640542001""",192,0.032413,2020-06-17 00:00:00,2020-06-17 00:00:00
"""0566549010""",100,0.026202,2019-06-04 00:00:00,2019-06-04 00:00:00
